In [12]:
import pandas as pd
from surprise import Dataset, Reader, KNNWithMeans

Testing
https://realpython.com/build-recommendation-engine-collaborative-filtering/

In [2]:
data = pd.read_csv("data/train.csv")

In [6]:
cp_affection= data[["customer_id","product_id"]].groupby(by=["customer_id","product_id"], as_index=False).value_counts().sort_values(by=["customer_id","count"])

print(cp_affection.shape)
print(len(cp_affection.customer_id.unique()), len(cp_affection.product_id.unique()))

# TODO: transactions may need to be scaled per user

cp_affection.rename(columns={"count": "times_bought"}, inplace=True)
cp_affection.head()

(47842, 3)
2000 100


,customer_id,product_id,times_bought
1,0,10,1
3,0,28,1
5,0,33,1
6,0,34,1
10,0,61,1


In [8]:
reader = Reader(rating_scale=(cp_affection.times_bought.min(), cp_affection.times_bought.max()))

In [10]:
rec_data = Dataset.load_from_df(cp_affection, reader)

In [13]:
# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

In [16]:
trainingSet = rec_data.build_full_trainset()

In [18]:
algo.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [32]:
customer_id, product_id = 0,0
prediction = algo.predict(customer_id, product_id)
prediction.est
prediction

Prediction(uid=0, iid=0, r_ui=None, est=3.0773778950213284, details={'actual_k': 22, 'was_impossible': False})

In [28]:
cp_affection[(cp_affection.customer_id==customer_id)&(cp_affection.product_id==product_id)]

,customer_id,product_id,times_bought


In [ ]:
# TODO: evaluate recommender